## Load data

In [1]:
file_test = 'C:\Data\Miroculus\A3_1_2016_text_inputs\A3_1_2016_binary_data_entities_normalize_rmstopwords_stem_test.tsv'
file_train = 'C:\Data\Miroculus\A3_1_2016_text_inputs\A3_1_2016_binary_data_entities_normalize_rmstopwords_stem_train.tsv'

In [2]:
import pandas as pd
test = pd.read_csv(file_test, quotechar='"', delimiter='\t', header=None, names=['Label', 'Text'])
train = pd.read_csv(file_train, quotechar='"', delimiter='\t', header=None, names=['Label', 'Text'])
test.head; train.head

<bound method NDFrame.head of              Label                                               Text
0      NO_RELATION  result identifi previous report breast cancer-...
1      NO_RELATION  conclus taken togeth mmvarentty1mm/ggvarentty1...
2         RELATION  revers correl mmvarentty1mm ggvarentty1gg also...
3         RELATION  result suggest recognit site mmvarentty1mm mmv...
4         RELATION  issu jci ghosh colleagu identifi uniqu microrn...
5      NO_RELATION  conclus mmvarentty1mm may promot cell ggvarent...
6      NO_RELATION        howev role mmvarentty1mm npc remain unknown
7         RELATION  targetscan predict ggvarentty1gg ggvarentty2gg...
8      NO_RELATION  increas mmvarentty1mm depend upon ggvarentty1g...
9      NO_RELATION  revers correl mmvarentty1mm ggvarentty1gg expr...
10        RELATION  studi identifi mmvarentty1mm mmvarentty2mm mic...
11        RELATION  show mmvarentty1mm oncosuppress microrna lost ...
12     NO_RELATION  newli identifi mmvarentty1mm/ggvarentty1

## Vectorise text data into numerical vector

### Using `CountVectorizer`

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,3), max_df=0.9, min_df=0.01)
train_tfidf_vectors = tfidf_vectorizer.fit_transform(train.Text)

** Note: ** Numbers not removed. Consider removing numbers.

`tfidf_vectorizer.vocabulary_` is a `dict`

In [4]:
tfidf_vectorizer.vocabulary_

{'001': 0,
 '05': 1,
 '16': 2,
 '3p': 3,
 '5p': 4,
 'act': 5,
 'activ': 6,
 'addit': 7,
 'affect': 8,
 'aim': 9,
 'alter': 10,
 'analys': 11,
 'analysi': 12,
 'analyz': 13,
 'apoptosi': 14,
 'arrest': 15,
 'assay': 16,
 'associ': 17,
 'bind': 18,
 'bioinformat': 19,
 'biomark': 20,
 'blot': 21,
 'breast': 22,
 'breast cancer': 23,
 'cancer': 24,
 'cancer cell': 25,
 'cancer tissu': 26,
 'carcinoma': 27,
 'caus': 28,
 'cell': 29,
 'cell cycl': 30,
 'cell growth': 31,
 'cell line': 32,
 'cell migrat': 33,
 'cell prolifer': 34,
 'chain': 35,
 'chain reaction': 36,
 'chang': 37,
 'clinic': 38,
 'cluster': 39,
 'colon': 40,
 'combin': 41,
 'compar': 42,
 'conclus': 43,
 'conclus mmvarentty1mm': 44,
 'confirm': 45,
 'contribut': 46,
 'control': 47,
 'correl': 48,
 'correl ggvarentty1gg': 49,
 'correl mmvarentty1mm': 50,
 'crc': 51,
 'cycl': 52,
 'data': 53,
 'decreas': 54,
 'decreas express': 55,
 'demonstr': 56,
 'demonstr mmvarentty1mm': 57,
 'depend': 58,
 'detect': 59,
 'determin': 60,
 

In [5]:
train_tfidf_vectors.shape

(10783, 358)

In [6]:
train_tfidf_vectors.data

array([ 0.29883483,  0.25380954,  0.2911042 , ...,  0.40269706,
        0.26405972,  0.38915471])

In [7]:
print(train_tfidf_vectors.nnz) # number of non-zero elements
print(train_tfidf_vectors.nnz / float(train_tfidf_vectors.shape[0])) # non-zero per sample
print(train_tfidf_vectors.nnz / float(train_tfidf_vectors.shape[0] * train_tfidf_vectors.shape[1]) * 100) # non-zero per feature space in %

134461
12.469720856904386
3.4831622505319517


## Data Preprocessing: 
### Attributes and Target

### Convert categorical labels into numerical labels

In [8]:
from sklearn import preprocessing

In [9]:
le = preprocessing.LabelEncoder()
le.fit(train.Label)
list(le.classes_)

['NO_RELATION', 'RELATION']

In [10]:
y_train = le.transform(train.Label);
y_test = le.transform(test.Label);

print(y_train[0:5])
print(le.inverse_transform(y_train[0:5]))

[0 0 1 1 1]
['NO_RELATION' 'NO_RELATION' 'RELATION' 'RELATION' 'RELATION']


`X` is data, `y` is target. `X` has the size of `n_samples` x `n_features`, `y` has `n_samples` x 1 labels

In [11]:
X_train = train_tfidf_vectors

In [12]:
X_test = tfidf_vectorizer.transform(test.Text)

In [13]:
X_test.shape

(3595, 358)

In [14]:
X_test.data

array([ 0.21861591,  0.23361039,  0.26788708, ...,  0.17149076,
        0.34588406,  0.18785396])

## Log Res: 

In [15]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(C=1e5, class_weight='balanced')
clf.fit(X_train, y_train)

LogisticRegression(C=100000.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [16]:
y_pred = clf.predict(X_test)

In [17]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.73      0.72      0.73      1804
          1       0.73      0.74      0.73      1791

avg / total       0.73      0.73      0.73      3595



In [18]:
print(metrics.confusion_matrix(y_test, y_pred))

[[1305  499]
 [ 471 1320]]


### Coeficients of the decision function The top word

In [19]:
clf.coef_

array([[  6.47870485e-01,  -5.35001545e-02,   2.69227915e+00,
          5.38008515e-01,  -7.17552906e-01,   5.49185436e-01,
          1.07412984e+00,   8.45464856e-01,   4.13784346e-01,
         -6.38260230e-01,   1.09848544e+00,   3.84878316e-01,
         -9.97783776e-02,   2.73146664e-01,   2.99123346e-01,
          3.51484906e-02,  -7.59126288e-02,  -5.49411545e-01,
          9.93956461e-01,   1.76878681e+00,  -9.66931306e-01,
          3.60339006e+00,  -2.17830146e-01,   8.34486975e-01,
         -1.71609621e+00,   2.14190536e-01,   1.53854953e+00,
         -1.31641192e+00,   1.33461469e+00,   3.20240550e-01,
          2.39186249e+00,   2.16732042e-01,   2.68432776e+00,
          7.31449722e-01,   3.71025960e-01,   2.46041160e+00,
         -4.62163527e+00,   2.31531532e-01,  -1.02328188e-01,
          1.17187547e+00,   9.08916473e-01,   1.04281853e+00,
         -5.71669558e-01,   4.32158535e-01,   7.23537651e-01,
          7.13534505e-01,   5.45971130e-01,   5.34864600e-01,
        

In [20]:
len(clf.coef_[0,])

358

Get the index of coef with the maximum value.

In [21]:
clf.coef_.argmax()

355

In [22]:
m = clf.coef_.argmax()
print(list(tfidf_vectorizer.vocabulary_.keys())[list(tfidf_vectorizer.vocabulary_.values()).index(m)])

western


Get the index of the highest 10 coef value.

**Note** `clf.coef_` is ndarray, see below:

In [23]:
clf.coef_[0]

array([  6.47870485e-01,  -5.35001545e-02,   2.69227915e+00,
         5.38008515e-01,  -7.17552906e-01,   5.49185436e-01,
         1.07412984e+00,   8.45464856e-01,   4.13784346e-01,
        -6.38260230e-01,   1.09848544e+00,   3.84878316e-01,
        -9.97783776e-02,   2.73146664e-01,   2.99123346e-01,
         3.51484906e-02,  -7.59126288e-02,  -5.49411545e-01,
         9.93956461e-01,   1.76878681e+00,  -9.66931306e-01,
         3.60339006e+00,  -2.17830146e-01,   8.34486975e-01,
        -1.71609621e+00,   2.14190536e-01,   1.53854953e+00,
        -1.31641192e+00,   1.33461469e+00,   3.20240550e-01,
         2.39186249e+00,   2.16732042e-01,   2.68432776e+00,
         7.31449722e-01,   3.71025960e-01,   2.46041160e+00,
        -4.62163527e+00,   2.31531532e-01,  -1.02328188e-01,
         1.17187547e+00,   9.08916473e-01,   1.04281853e+00,
        -5.71669558e-01,   4.32158535e-01,   7.23537651e-01,
         7.13534505e-01,   5.45971130e-01,   5.34864600e-01,
         9.86101527e-02,

In [24]:
print(clf.coef_[0][0])
print(clf.coef_[0][1])

0.647870485388
-0.0535001544978


In [25]:
import numpy as np
topN = np.argsort(clf.coef_[0])[::-1][:20]
print(topN)

[355 162  21 328 264 117 103 217 105 323 221 245 331  67   2  32 287 330
 288  35]


In [26]:
for v in topN:
    print(list(tfidf_vectorizer.vocabulary_.keys())[list(tfidf_vectorizer.vocabulary_.values()).index(v)])


western
invers
blot
target ggvarentty1gg
prostat
ggvarentty2gg
gene ggvarentty1gg
mmvarentty2mm mmvarentty3mm
ggvarentty1gg
taken
mmvarentty3mm mmvarentty4mm mmvarentty5mm
partial
target mmvarentty1mm
directli
16
cell line
report assay
target ggvarentty2gg
repress
chain
